In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')

In [ ]:
df.head()

In [ ]:
df.iloc[0,1]

In [ ]:
df.iloc[100,2:]

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
#TextVectorization??

In [ ]:
df[df.columns[2:]].values

In [ ]:
df.iloc[:,2:].values

In [ ]:
x = df['comment_text']
y = df.iloc[:,2:].values

In [ ]:
MAX_WORDS = 200000 #no of words in vocab

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_WORDS, output_sequence_length=1800, output_mode='int')

In [ ]:
x.values

In [ ]:
vectorizer.adapt(x.values)

In [ ]:
vectorizer('Hello world, How are you doing?')[:10]

In [ ]:
vectorizer_text = vectorizer(x.values)

In [ ]:
vectorizer_text

In [ ]:
#MCSHBAP(data pipeline generation) -map, cache, shuffle, batch, prefetch
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [ ]:
batch_x,batch_y = dataset.as_numpy_iterator().next()

In [ ]:
batch_x.shape

In [ ]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [ ]:
# from sklearn.model_selection import train_test_split

# x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.3, random_state=42)
# x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.75, random_state=42)


In [ ]:
train_generator = train.as_numpy_iterator()

In [ ]:
train_generator.next() # for every run it goes to the next batch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding #Dropout-method for regularization

In [ ]:
model = Sequential()
#embeddding layer
model.add(Embedding(MAX_WORDS+1, 32))
#Bidirectional LSTM layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature Extractor Fully connected layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

In [ ]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [ ]:
model.summary()

In [ ]:
history = model.fit(train, epochs=1, validation_data=val)

In [ ]:
from matplotlib.pyplot as plt
plt.figure(figsize=(14,8))
pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
input_text = vectorizer('You arrogant punk! He did not shoot the bullet')

In [ ]:
res = model.predict(input_text)

In [ ]:
(res>0.5).astype(int)

In [ ]:
bacth_x, batch_y = test.as_numpy_iterator().next()

In [ ]:
(model.predict(batch_x)>0.5).astype(int)

In [ ]:
res.shape

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()


In [ ]:
for batch in test.as_numpy_iterator():
    x_true, y_true = batch
    yhat = model.predict(x_true)
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    pre.update_state(y_true,y_hat)
    re.update_state(y_true,y_hat)    
    acc.update_state(y_true,y_hat)    

In [ ]:
print(f'Precision: {pre.result().numpy()},Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')
